# 6. Modelos de Regressão


Este notebook explora diferentes modelos de regressão para análise de dados, incluindo Regressão Linear e Lasso, utilizando técnicas de pré-processamento como transformação Box-Cox e seleção de variáveis. O objetivo é avaliar o desempenho dos modelos em termos de métricas como RMSE e R², tanto nos datasets completos quanto após a remoção de outliers das questões do Enem (vetorizadas).


In [96]:
# Importando Dependências para Modelos de Regressão
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LassoCV

In [97]:
# Leitura dos dados
enem_data = pd.read_pickle("../data/final/enem_data_final.pkl")
enem_data.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,...,enunciado_embbedings_word2vec_50,gabarito_embbedings_word2vec_50,distratores_embbedings_word2vec_50,similaridade_enunciado_gabarito_50,similaridade_enunciado_distratores_50,similaridade_gabarito_distratores_50,resposta_llama,acerto_llamma,resposta_deepseek,acerto_deepseek
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,...,"[0.09772969, 0.013166125, -0.032748703, 0.0580...","[-0.04434625, 0.2105245, -0.18144351, 0.030336...","[0.12940213, 0.22066694, 0.0586984, 0.03139533...",0.251504,0.736054,0.426387,C,True,C,True
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,...,"[-0.018823402, 0.07832528, -0.016192827, 0.027...","[0.21008351, 0.5248949, 0.03625112, 0.01772637...","[0.078055575, 0.19092263, -0.0076030395, -0.03...",0.634785,0.504648,0.587013,D,True,D,True
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,...,"[-0.013062097, 0.038470805, -0.020605344, 0.05...","[0.16413634, -0.035050992, 0.08454134, 0.23227...","[0.1313074, 0.058539692, 0.0961813, 0.17813754...",0.655402,0.634760,0.763059,D,True,D,True
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,...,"[-0.042833664, -0.01434522, -0.002431296, 0.01...","[0.102909006, 0.06116425, -0.13846825, 0.33034...","[0.11364358, 0.071867414, 0.046681672, 0.18046...",0.485125,0.628863,0.766145,E,True,E,True
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,...,"[0.010493249, 0.11095436, 0.017243173, 0.05319...","[0.267512, -0.081234336, 0.08019033, 0.1294613...","[0.13863416, 0.16260158, -0.0043905806, 0.2472...",0.559584,0.707497,0.692311,C,True,C,True


In [98]:
enem_data = enem_data[enem_data["nu_param_B"].notna()]

In [99]:
enem_data["similaridade_enunciado_gabarito_100"][0]

0.23648179

---

## 6.1. Embeddings de 300 Dimensões


In [100]:
# Coletando os dados
X_300 = [
    np.array(embedding) for embedding in enem_data["enunciado_embbedings_word2vec_300"]
]  # Somente Embeddings 300

y_300 = enem_data["nu_param_B"]  # Parâmetro de Dificuldade (B)

In [101]:
# Aplicando Transformações
add_list = [(y_300.min() * (-1)) + 1] * len(y_300)
y_300 = y_300 + add_list

# Aplicando Box-Cox
y_300, best_lambda = stats.boxcox(y_300)
print(best_lambda)

0.6304389864783265


### 6.1.1. Regressão Linear


In [102]:
# Separando em conjunto de treino e teste
X_train_300, X_test_300, y_train_300, y_test_300 = train_test_split(
    X_300, y_300, test_size=0.3, random_state=42
)

In [103]:
# Adicionando constante
X_train_300 = sm.add_constant(X_train_300)
X_test_300 = sm.add_constant(X_test_300)

In [104]:
# Criando o modelo e realizando predição
linear_model_300 = sm.OLS(y_train_300, X_train_300).fit()
prediction_linear_model_300 = linear_model_300.predict(X_test_300)

In [105]:
# Cálculo do RMSE
rmse_300 = root_mean_squared_error(y_test_300, prediction_linear_model_300)
print("RMSE: ", rmse_300)

RMSE:  0.7720791146717553


In [106]:
# Visualização do modelo e resultados
print(linear_model_300.summary(alpha=0.05))

/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 15 Jun 2025   Prob (F-statistic):                nan
Time:                        17:26:44   Log-Likelihood:                 5970.9
No. Observations:                 186   AIC:                        -1.157e+04
Df Residuals:                       0   BIC:                        -1.097e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2107        inf          0        n

### 6.1.2. Regressão Lasso


In [107]:
# Separando em conjunto de treino e teste
X_train_300_lasso, X_test_300_lasso, y_train_300_lasso, y_test_300_lasso = (
    train_test_split(X_300, y_300, test_size=0.3, random_state=42)
)

In [108]:
lasso_model_300 = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0).fit(
    X_train_300_lasso, y_train_300_lasso
)
pred_lasso_model_300 = lasso_model_300.predict(X_test_300_lasso)

print("Melhor R2 Score:", lasso_model_300.score(X_train_300_lasso, y_train_300_lasso))
print("Melhor Alpha:", lasso_model_300.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_300_lasso, pred_lasso_model_300),
)

Melhor R2 Score: 0.5344153222227536
Melhor Alpha: 0.001
RMSE com o alpha escolhido: 0.5177163330655292


### 6.1.3. Regressões com Corte (-3,3)


In [109]:
# Aplicando corte de dificuldade sugerido pelo artigo motivador (-3, 3)
enem_filtered = enem_data.copy()

enem_filtered = enem_filtered[
    (enem_filtered["nu_param_B"] >= -3) & (enem_filtered["nu_param_B"] <= 3)
]
enem_filtered["nu_param_B"].describe()

count    263.000000
mean       1.075595
std        0.661348
min       -0.726450
25%        0.612925
50%        1.051180
75%        1.535350
max        2.802770
Name: nu_param_B, dtype: float64

In [110]:
# Coletando os dados
X_300_filtered = [
    np.array(embedding)
    for embedding in enem_filtered["enunciado_embbedings_word2vec_300"]
]
y_300_filtered = enem_filtered["nu_param_B"]

In [111]:
# Aplicando Transformações
add_list = [(y_300_filtered.min() * (-1)) + 1] * len(y_300_filtered)
y_300_filtered = y_300_filtered + add_list

# Aplicando Boxcox
y_300_filtered, best_lambda = stats.boxcox(y_300_filtered)
print(best_lambda)

0.9332701631134968


### 6.1.3.A) Regressão Linear


In [112]:
# Dividindo os dados em treino e teste
X_train_300_filtered, X_test_300_filtered, y_train_300_filtered, y_test_300_filtered = (
    train_test_split(X_300_filtered, y_300_filtered, test_size=0.3, random_state=42)
)

In [113]:
# Adicionando constante
X_train_300_filtered = sm.add_constant(X_train_300_filtered)
X_test_300_filtered = sm.add_constant(X_test_300_filtered)

In [114]:
# Criando o modelo e realizando a predição
linear_model_300_filtered = sm.OLS(y_train_300_filtered, X_train_300_filtered).fit()
pred_linear_model_300_filtered = linear_model_300_filtered.predict(X_test_300_filtered)

In [115]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_300_filtered, pred_linear_model_300_filtered)
print("RMSE:", rms)

RMSE: 1.2188517275533708


In [116]:
print(linear_model_300_filtered.summary(alpha=0.05))

/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 15 Jun 2025   Prob (F-statistic):                nan
Time:                        17:26:45   Log-Likelihood:                 5833.8
No. Observations:                 184   AIC:                        -1.130e+04
Df Residuals:                       0   BIC:                        -1.071e+04
Df Model:                         183                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4067        inf          0        n

### 6.1.3.B) Regressão Lasso


In [117]:
# Dividindo os dados em treino e teste
(
    X_train_300_filtered_lasso,
    X_test_300_filtered_lasso,
    y_train_300_filtered_lasso,
    y_test_300_filtered_lasso,
) = train_test_split(X_300_filtered, y_300_filtered, test_size=0.3, random_state=42)

In [118]:
lasso_model_300_filtered = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100], random_state=0, tol=0.1
).fit(X_train_300_filtered_lasso, y_train_300_filtered_lasso)

pred_300_lasso_filtered = lasso_model_300_filtered.predict(X_test_300_filtered_lasso)

print(
    "Melhor R2 score:",
    lasso_model_300_filtered.score(
        X_train_300_filtered_lasso, y_train_300_filtered_lasso
    ),
)
print("Melhor Alpha:", lasso_model_300_filtered.alpha_)
print(
    "RMSE com o Alpha escolhido:",
    root_mean_squared_error(y_test_300_filtered_lasso, pred_300_lasso_filtered),
)

Melhor R2 score: 0.5662683125013057
Melhor Alpha: 0.001
RMSE com o Alpha escolhido: 0.6342396428478064


### 6.1.4. LLM & Similaridade Features


In [119]:
# Coletando os dados
y_300_all_features = enem_data["nu_param_B"]
X_300_all_features = [
    np.concatenate(
        [
            np.array(embedding),
            [
                acerto_deepseek,
                acerto_llamma,
                similaridade_enunciado_gabarito_300,
                similaridade_enunciado_distratores_300,
                similaridade_gabarito_distratores_300,
            ],
        ]
    )
    for acerto_deepseek, acerto_llamma, embedding, similaridade_enunciado_gabarito_300, similaridade_enunciado_distratores_300, similaridade_gabarito_distratores_300 in zip(
        enem_data["acerto_deepseek"],
        enem_data["acerto_llamma"],
        enem_data["enunciado_embbedings_word2vec_300"],
        enem_data["similaridade_enunciado_gabarito_300"],
        enem_data["similaridade_enunciado_distratores_300"],
        enem_data["similaridade_gabarito_distratores_300"],
    )
]

In [120]:
# Aplicando Transformações
add_list = [(y_300_all_features.min() * (-1)) + 1] * len(y_300_all_features)
y_300_all_features = y_300_all_features + add_list

# Aplicando Boxcox
y_300_all_features, best_lambda = stats.boxcox(y_300_all_features)
print(best_lambda)

0.6304389864783265


### 6.1.4.A) Regressão linear


In [121]:
# Separando em conjunto de treino e teste

(
    X_train_300_all_features,
    X_test_300_all_features,
    y_train_300_all_features,
    y_test_300_all_features,
) = train_test_split(
    X_300_all_features, y_300_all_features, test_size=0.3, random_state=42
)

In [122]:
# Adicionando constante
X_train_300_all_features = sm.add_constant(X_train_300_all_features)
X_test_300_all_features = sm.add_constant(X_test_300_all_features)

In [123]:
# Criando o modelo e realizando predição
linear_model_300_all_features = sm.OLS(
    y_train_300_all_features, X_train_300_all_features
).fit()
pred_linear_model_300_all_features = linear_model_300_all_features.predict(
    X_test_300_all_features
)

In [124]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_300_all_features, pred_linear_model_300_all_features)
print("RMSE", rms)

RMSE 0.7478685280228781


In [125]:
print(linear_model_300_all_features.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 15 Jun 2025   Prob (F-statistic):                nan
Time:                        17:26:48   Log-Likelihood:                 5886.8
No. Observations:                 186   AIC:                        -1.140e+04
Df Residuals:                       0   BIC:                        -1.080e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0449        inf          0        n

/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


### 6.1.4.B) Lasso


In [126]:
# Separando em conjunto de treino e teste
(
    X_train_300_lasso_all_features,
    X_test_300_lasso_all_features,
    y_train_300_lasso_all_features,
    y_test_300_lasso_all_features,
) = train_test_split(
    X_300_all_features, y_300_all_features, test_size=0.3, random_state=42
)

In [127]:
lasso_model_all_features_300 = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0
).fit(X_train_300_lasso_all_features, y_train_300_lasso_all_features)

pred_300_lasso_all_features = lasso_model_all_features_300.predict(
    X_test_300_lasso_all_features
)

print(
    "melhor r2 score:",
    lasso_model_all_features_300.score(
        X_train_300_lasso_all_features, y_train_300_lasso_all_features
    ),
)
print("melhor alpha:", lasso_model_all_features_300.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_300_lasso_all_features, pred_300_lasso_all_features),
)

melhor r2 score: 0.547574748756191
melhor alpha: 0.001
RMSE com o alpha escolhido: 0.48813975574107166


---

## 6.2. Embeddings de 100 Dimensões


In [128]:
# Coletando os dados
X_100 = [
    np.array(embedding) for embedding in enem_data["enunciado_embbedings_word2vec_100"]
]  # Somente Embeddings

y_100 = enem_data["nu_param_B"]  # Parâmetro de Dificuldade (B)

In [129]:
# Aplicando Transformações
add_list = [(y_100.min() * (-1)) + 1] * len(y_100)
y_100 = y_100 + add_list

# Aplicando Box-Cox
y_100, best_lambda = stats.boxcox(y_100)
print(best_lambda)

0.6304389864783265


### 6.2.1. Regressão Linear


In [130]:
# Separando em conjunto de treino e teste
X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(
    X_100, y_100, test_size=0.3, random_state=42
)

In [131]:
# Adicionando constante
X_train_100 = sm.add_constant(X_train_100)
X_test_100 = sm.add_constant(X_test_100)

In [132]:
# Criando o modelo e realizando predição
linear_model_100 = sm.OLS(y_train_100, X_train_100).fit()
pred_linear_model_100 = linear_model_100.predict(X_test_100)

In [133]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_100, pred_linear_model_100)
print("RMSE: ", rms)

RMSE:  0.7525703952360024


In [134]:
# Visualização do modelo e resultados
print(linear_model_100.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.184
Method:                 Least Squares   F-statistic:                     1.417
Date:                Sun, 15 Jun 2025   Prob (F-statistic):             0.0498
Time:                        17:26:49   Log-Likelihood:                -37.361
No. Observations:                 186   AIC:                             276.7
Df Residuals:                      85   BIC:                             602.5
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9890      0.915      2.173      0.0

### 6.2.2. Regressão Lasso


In [135]:
# Separando em conjunto de treino e teste
X_train_100_lasso, X_test_100_lasso, y_train_100_lasso, y_test_100_lasso = (
    train_test_split(X_100, y_100, test_size=0.3, random_state=42)
)

In [136]:
model_lasso = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0).fit(
    X_train_100_lasso, y_train_100_lasso
)
pred_lasso = model_lasso.predict(X_test_100_lasso)

print("Melhor R2 Score:", model_lasso.score(X_train_100_lasso, y_train_100_lasso))
print("Melhor Alpha:", model_lasso.alpha_)
print(
    "RMSE com o alpha escolhido:", root_mean_squared_error(y_test_100_lasso, pred_lasso)
)

Melhor R2 Score: 0.0
Melhor Alpha: 10.0
RMSE com o alpha escolhido: 0.49362604239339286


### 6.2.3. Regressões com Corte (-3,3)


In [137]:
# Aplicando corte de dificuldade sugerido pelo artigo motivador (-3, 3)
enem_filtered = enem_data.copy()

enem_filtered = enem_filtered[
    (enem_filtered["nu_param_B"] >= -3) & (enem_filtered["nu_param_B"] <= 3)
]
enem_filtered["nu_param_B"].describe()

count    263.000000
mean       1.075595
std        0.661348
min       -0.726450
25%        0.612925
50%        1.051180
75%        1.535350
max        2.802770
Name: nu_param_B, dtype: float64

In [138]:
# Coletando os dados
X_100_filtered = [
    np.array(embedding)
    for embedding in enem_filtered["enunciado_embbedings_word2vec_100"]
]
y_100_filtered = enem_filtered["nu_param_B"]

In [139]:
# Aplicando Transformações
add_list = [(y_100_filtered.min() * (-1)) + 1] * len(y_100_filtered)
y_100_filtered = y_100_filtered + add_list

# Aplicando Boxcox
y_100_filtered, best_lambda = stats.boxcox(y_100_filtered)
print(best_lambda)

0.9332701631134968


### 6.2.3.A) Regressão Linear


In [140]:
# Dividindo os dados em treino e teste
X_train_100_filtered, X_test_100_filtered, y_train_100_filtered, y_test_100_filtered = (
    train_test_split(X_100_filtered, y_100_filtered, test_size=0.3, random_state=42)
)

In [141]:
# Adicionando constante
X_train_100_filtered = sm.add_constant(X_train_100_filtered)
X_test_100_filtered = sm.add_constant(X_test_100_filtered)

In [142]:
# Criando o modelo e realizando a predição
linear_model_100_filtered = sm.OLS(y_train_100_filtered, X_train_100_filtered).fit()
pred_linear_model_100_filtered = linear_model_100_filtered.predict(X_test_100_filtered)

In [143]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_100_filtered, pred_linear_model_100_filtered)
print("RMSE:", rms)

RMSE: 1.1116035092062138


In [144]:
print(linear_model_100_filtered.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.639
Model:                            OLS   Adj. R-squared:                  0.204
Method:                 Least Squares   F-statistic:                     1.468
Date:                Sun, 15 Jun 2025   Prob (F-statistic):             0.0360
Time:                        17:26:50   Log-Likelihood:                -79.988
No. Observations:                 184   AIC:                             362.0
Df Residuals:                      83   BIC:                             686.7
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.4525      1.241      2.782      0.0

### 6.2.3.B) Regressão Lasso


In [145]:
# Dividindo os dados em treino e teste
(
    X_train_100_filtered_lasso,
    X_test_100_filtered_lasso,
    y_train_100_filtered_lasso,
    y_test_100_filtered_lasso,
) = train_test_split(X_100_filtered, y_100_filtered, test_size=0.3, random_state=42)

In [146]:
lasso_model_100_filtered = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100], random_state=0, tol=0.1
).fit(X_train_100_filtered_lasso, y_train_100_filtered_lasso)

pred_lasso_filtered_100 = lasso_model_100_filtered.predict(X_test_100_filtered_lasso)

print(
    "Melhor R2 score:",
    lasso_model_100_filtered.score(
        X_train_100_filtered_lasso, y_train_100_filtered_lasso
    ),
)
print("Melhor Alpha:", lasso_model_100_filtered.alpha_)
print(
    "RMSE com o Alpha escolhido:",
    root_mean_squared_error(y_test_100_filtered_lasso, pred_lasso_filtered_100),
)

Melhor R2 score: 0.4089702399339846
Melhor Alpha: 0.001
RMSE com o Alpha escolhido: 0.6878739383423418


### 6.2.4. LLM & Similaridade Features


In [147]:
# Coletando os dados
y_100_all_features = enem_data["nu_param_B"]
X_100_all_features = [
    np.concatenate(
        [
            np.array(embedding),
            [
                acerto_deepseek,
                acerto_llamma,
                similaridade_enunciado_gabarito_100,
                similaridade_enunciado_distratores_100,
                similaridade_gabarito_distratores_100,
            ],
        ]
    )
    for acerto_deepseek, acerto_llamma, embedding, similaridade_enunciado_gabarito_100, similaridade_enunciado_distratores_100, similaridade_gabarito_distratores_100 in zip(
        enem_data["acerto_deepseek"],
        enem_data["acerto_llamma"],
        enem_data["enunciado_embbedings_word2vec_100"],
        enem_data["similaridade_enunciado_gabarito_100"],
        enem_data["similaridade_enunciado_distratores_100"],
        enem_data["similaridade_gabarito_distratores_100"],
    )
]

In [148]:
# Aplicando Transformações
add_list = [(y_100_all_features.min() * (-1)) + 1] * len(y_100_all_features)
y_100_all_features = y_100_all_features + add_list

# Aplicando Boxcox
y_100_all_features, best_lambda = stats.boxcox(y_100_all_features)
print(best_lambda)

0.6304389864783265


### 6.2.4.A) Regressão linear


In [149]:
# Separando em conjunto de treino e teste

(
    X_train_100_all_features,
    X_test_100_all_features,
    y_train_100_all_features,
    y_test_100_all_features,
) = train_test_split(
    X_100_all_features, y_100_all_features, test_size=0.3, random_state=42
)

In [150]:
# Adicionando constante
X_train_100_all_features = sm.add_constant(X_train_100_all_features)
X_test_100_all_features = sm.add_constant(X_test_100_all_features)

In [151]:
# Criando o modelo e realizando predição
linear_model_100_all_features = sm.OLS(
    y_train_100_all_features, X_train_100_all_features
).fit()
pred_linear_model_100_all_features = linear_model_100_all_features.predict(
    X_test_100_all_features
)

In [152]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_100_all_features, pred_linear_model_100_all_features)
print("RMSE", rms)

RMSE 0.7301775694828514


In [153]:
print(linear_model_100_all_features.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.639
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                     1.347
Date:                Sun, 15 Jun 2025   Prob (F-statistic):             0.0817
Time:                        17:26:51   Log-Likelihood:                -33.892
No. Observations:                 186   AIC:                             279.8
Df Residuals:                      80   BIC:                             621.7
Df Model:                         105                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9024      0.964      1.974      0.0

### 6.2.4.B) Lasso


In [154]:
# Separando em conjunto de treino e teste
(
    X_train_100_lasso_all_features,
    X_test_100_lasso_all_features,
    y_train_100_lasso_all_features,
    y_test_100_lasso_all_features,
) = train_test_split(
    X_100_all_features, y_100_all_features, test_size=0.3, random_state=42
)

In [155]:
lasso_model_all_features_100 = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0
).fit(X_train_100_lasso_all_features, y_train_100_lasso_all_features)

pred_100_lasso_all_features = lasso_model_all_features_100.predict(
    X_test_100_lasso_all_features
)

print(
    "melhor r2 score:",
    lasso_model_all_features_100.score(
        X_train_100_lasso_all_features, y_train_100_lasso_all_features
    ),
)
print("melhor alpha:", lasso_model_all_features_100.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_100_lasso_all_features, pred_100_lasso_all_features),
)

melhor r2 score: 0.0
melhor alpha: 10.0
RMSE com o alpha escolhido: 0.49362604239339286


---

## 6.3. Embeddings de 50 Dimensões


In [156]:
# Coletando os dados
X_50 = [
    np.array(embedding) for embedding in enem_data["enunciado_embbedings_word2vec_50"]
]  # Somente Embeddings 50

y_50 = enem_data["nu_param_B"]  # Parâmetro de Dificuldade (B)

In [157]:
# Aplicando Transformações
add_list = [(y_50.min() * (-1)) + 1] * len(y_50)
y_50 = y_50 + add_list

# Aplicando Box-Cox
y_50, best_lambda = stats.boxcox(y_50)
print(best_lambda)

0.6304389864783265


### 6.3.1. Regressão Linear


In [158]:
# Separando em conjunto de treino e teste
X_train_50, X_test_50, y_train_50, y_test_50 = train_test_split(
    X_50, y_50, test_size=0.3, random_state=42
)

In [159]:
# Adicionando constante
X_train_50 = sm.add_constant(X_train_50)
X_test_50 = sm.add_constant(X_test_50)

In [160]:
# Criando o modelo e realizando predição
linear_model_50 = sm.OLS(y_train_50, X_train_50).fit()
pred_linear_model_50 = linear_model_50.predict(X_test_50)

In [161]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_50, pred_linear_model_50)
print("RMSE: ", rms)

RMSE:  0.5394759971562638


In [162]:
# Visualização do modelo e resultados
print(linear_model_50.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.316
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     1.245
Date:                Sun, 15 Jun 2025   Prob (F-statistic):              0.162
Time:                        17:26:52   Log-Likelihood:                -93.299
No. Observations:                 186   AIC:                             288.6
Df Residuals:                     135   BIC:                             453.1
Df Model:                          50                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5116      0.659      2.292      0.0

### 6.3.2. Regressão Lasso


In [163]:
# Separando em conjunto de treino e teste
X_train_50_lasso, X_test_50_lasso, y_train_50_lasso, y_test_50_lasso = train_test_split(
    X_50, y_50, test_size=0.3, random_state=42
)

In [164]:
lasso_model_50 = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0).fit(
    X_train_50_lasso, y_train_50_lasso
)
pred_lasso_model_50 = lasso_model_50.predict(X_test_50_lasso)

print("Melhor R2 Score:", lasso_model_50.score(X_train_50_lasso, y_train_50_lasso))
print("Melhor Alpha:", lasso_model_50.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_50_lasso, pred_lasso_model_50),
)

Melhor R2 Score: 0.0
Melhor Alpha: 10.0
RMSE com o alpha escolhido: 0.49362604239339286


### 6.3.3. Regressões com Corte (-3,3)


In [165]:
# Aplicando corte de dificuldade sugerido pelo artigo motivador (-3, 3)
enem_filtered = enem_data.copy()

enem_filtered = enem_filtered[
    (enem_filtered["nu_param_B"] >= -3) & (enem_filtered["nu_param_B"] <= 3)
]
enem_filtered["nu_param_B"].describe()

count    263.000000
mean       1.075595
std        0.661348
min       -0.726450
25%        0.612925
50%        1.051180
75%        1.535350
max        2.802770
Name: nu_param_B, dtype: float64

In [166]:
# Coletando os dados
X_50_filtered = [
    np.array(embedding)
    for embedding in enem_filtered["enunciado_embbedings_word2vec_50"]
]
y_50_filtered = enem_filtered["nu_param_B"]

In [167]:
# Aplicando Transformações
add_list = [(y_50_filtered.min() * (-1)) + 1] * len(y_50_filtered)
y_50_filtered = y_50_filtered + add_list

# Aplicando Boxcox
y_50_filtered, best_lambda = stats.boxcox(y_50_filtered)
print(best_lambda)

0.9332701631134968


### 6.3.3.A) Regressão Linear


In [168]:
# Dividindo os dados em treino e teste
X_train_50_filtered, X_test_50_filtered, y_train_50_filtered, y_test_50_filtered = (
    train_test_split(X_50_filtered, y_50_filtered, test_size=0.3, random_state=42)
)

In [169]:
# Adicionando constante
X_train_50_filtered = sm.add_constant(X_train_50_filtered)
X_test_50_filtered = sm.add_constant(X_test_50_filtered)

In [170]:
# Criando o modelo e realizando a predição
linear_model_50_filtered = sm.OLS(y_train_50_filtered, X_train_50_filtered).fit()
pred_linear_model_50_filtered = linear_model_50_filtered.predict(X_test_50_filtered)

In [171]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_50_filtered, pred_linear_model_50_filtered)
print("RMSE:", rms)

RMSE: 0.7040126990763843


In [172]:
print(linear_model_50_filtered.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     1.136
Date:                Sun, 15 Jun 2025   Prob (F-statistic):              0.281
Time:                        17:26:53   Log-Likelihood:                -140.96
No. Observations:                 184   AIC:                             383.9
Df Residuals:                     133   BIC:                             547.9
Df Model:                          50                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4396      0.858      1.679      0.0

### 6.3.3.B) Regressão Lasso


In [173]:
# Dividindo os dados em treino e teste
(
    X_train_50_filtered_lasso,
    X_test_50_filtered_lasso,
    y_train_50_filtered_lasso,
    y_test_50_filtered_lasso,
) = train_test_split(X_50_filtered, y_50_filtered, test_size=0.3, random_state=42)

In [174]:
lasso_model_50_filtered = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100], random_state=0, tol=0.1
).fit(X_train_50_filtered_lasso, y_train_50_filtered_lasso)

pred_lasso_model_50_filtered = lasso_model_50_filtered.predict(X_test_50_filtered_lasso)

print(
    "Melhor R2 score:",
    lasso_model_50_filtered.score(X_train_50_filtered_lasso, y_train_50_filtered_lasso),
)
print("Melhor Alpha:", lasso_model_50_filtered.alpha_)
print(
    "RMSE com o Alpha escolhido:",
    root_mean_squared_error(y_test_50_filtered_lasso, pred_lasso_model_50_filtered),
)

Melhor R2 score: 0.0
Melhor Alpha: 100.0
RMSE com o Alpha escolhido: 0.608577671186741


### 6.3.4. LLM & Similaridade Features


In [175]:
# Coletando os dados
y_50_all_features = enem_data["nu_param_B"]
X_50_all_features = [
    np.concatenate(
        [
            np.array(embedding),
            [
                acerto_deepseek,
                acerto_llamma,
                similaridade_enunciado_gabarito_50,
                similaridade_enunciado_distratores_50,
                similaridade_gabarito_distratores_50,
            ],
        ]
    )
    for acerto_deepseek, acerto_llamma, embedding, similaridade_enunciado_gabarito_50, similaridade_enunciado_distratores_50, similaridade_gabarito_distratores_50 in zip(
        enem_data["acerto_deepseek"],
        enem_data["acerto_llamma"],
        enem_data["enunciado_embbedings_word2vec_50"],
        enem_data["similaridade_enunciado_gabarito_50"],
        enem_data["similaridade_enunciado_distratores_50"],
        enem_data["similaridade_gabarito_distratores_50"],
    )
]

In [176]:
# Aplicando Transformações
add_list = [(y_50_all_features.min() * (-1)) + 1] * len(y_50_all_features)
y_50_all_features = y_50_all_features + add_list

# Aplicando Boxcox
y_50_all_features, best_lambda = stats.boxcox(y_50_all_features)
print(best_lambda)

0.6304389864783265


### 6.3.4.A) Regressão linear


In [177]:
# Separando em conjunto de treino e teste

(
    X_train_50_all_features,
    X_test_50_all_features,
    y_train_50_all_features,
    y_test_50_all_features,
) = train_test_split(
    X_50_all_features, y_50_all_features, test_size=0.3, random_state=42
)

In [178]:
# Adicionando constante
X_train_50_all_features = sm.add_constant(X_train_50_all_features)
X_test_50_all_features = sm.add_constant(X_test_50_all_features)

In [179]:
# Criando o modelo e realizando predição
linear_model_50_all_features = sm.OLS(
    y_train_50_all_features, X_train_50_all_features
).fit()
pred_linear_model_50_all_features = linear_model_50_all_features.predict(
    X_test_50_all_features
)

In [180]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_50_all_features, pred_linear_model_50_all_features)
print("RMSE", rms)

RMSE 0.5409367161926952


In [181]:
print(linear_model_50_all_features.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     1.423
Date:                Sun, 15 Jun 2025   Prob (F-statistic):             0.0536
Time:                        17:26:53   Log-Likelihood:                -84.746
No. Observations:                 186   AIC:                             281.5
Df Residuals:                     130   BIC:                             462.1
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.7942      0.697      2.576      0.0

### 6.3.4.B) Lasso


In [182]:
# Separando em conjunto de treino e teste
(
    X_train_50_lasso_all_features,
    X_test_50_lasso_all_features,
    y_train_50_lasso_all_features,
    y_test_50_lasso_all_features,
) = train_test_split(
    X_50_all_features, y_50_all_features, test_size=0.3, random_state=42
)

In [183]:
lasso_model_all_features_50 = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0
).fit(X_train_50_lasso_all_features, y_train_50_lasso_all_features)

pred_50_lasso_all_features = lasso_model_all_features_50.predict(
    X_test_50_lasso_all_features
)

print(
    "melhor r2 score:",
    lasso_model_all_features_50.score(
        X_train_50_lasso_all_features, y_train_50_lasso_all_features
    ),
)
print("melhor alpha:", lasso_model_all_features_50.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_50_lasso_all_features, pred_50_lasso_all_features),
)

melhor r2 score: 0.0
melhor alpha: 10.0
RMSE com o alpha escolhido: 0.49362604239339286


---
